In [1]:
import numpy as np
np.random.seed(10)
import tensorflow as tf
tf.random.set_seed(10)
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ReduceLROnPlateau,EarlyStopping
import matplotlib.pyplot as plt

In [2]:
import glob

In [3]:
# In physical space use 'raw_train.reshape(-1,121,281)' to visualize original
num_snapshots = 12564
#directory = '/lcrc/project/AIEADA-2/era5_data/full_data/snapshots/'
directory = '/lus/grand/projects/datascience/bethanyl/AIEADA/snapshots/'

#var_list = [raw_train_z500,
#            raw_train_u250,raw_train_v250,raw_train_t250,
#            raw_train_u850,raw_train_v850,raw_train_t850,
#            raw_train_blh,raw_train_tcwv]
input_window = 14
output_window = 7

In [4]:
def load_file_npy(file_path):
    file_path_string = bytes.decode(file_path.numpy())
    data = np.load(file_path_string)
    return data

In [5]:
def read_npy_file(file_path):
    file_path_string = bytes.decode(file_path.numpy())
    data = np.load(file_path_string)
    return data.astype(np.float32)

In [6]:
def parse_function(file_path):
    #example = tf.py_function(load_file_npy, inp=file_path, Tout=tf.float32)
    example = tf.py_function(read_npy_file, [file_path], tf.float32)
    #example = tf.py_function(np.load, file_path, tf.string)
    #tf.print(file_path)
    #example = tf.io.read_file(file_path)
    #example = tf.io.decode_raw(example, tf.float32)
    print(example)
    input_window = 14
    input_data = example[:input_window,:]
    output_data = example[input_window:,:]
    #return input_data, {'pred':output_data, 'recon':input_data}
    return input_data, (output_data, input_data)

In [7]:
def build_dataset(batch_size):
    dataset_dir = directory + 'split_examples/train_data/'
    filelist = glob.glob(dataset_dir + '*.npy')
    train_dataset = tf.data.Dataset.from_tensor_slices(filelist)
    shuffle_buffer = 6000
    train_dataset = train_dataset.shuffle(shuffle_buffer,reshuffle_each_iteration=True)
    train_dataset = train_dataset.map(parse_function, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    #train_dataset = train_dataset.map(
    #    lambda item: tuple(tf.py_function(read_npy_file, [item], [tf.float32,])))
    train_dataset = train_dataset.batch(batch_size)
    train_dataset = train_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    
    return train_dataset

In [8]:
example = np.load(directory + 'split_examples/train_data/example151.npy')

In [9]:
print(example.shape)
print(example.dtype)

(21, 38979)
float32


In [10]:
batch_size = 512
train_dataset = build_dataset(batch_size)

Tensor("EagerPyFunc:0", dtype=float32, device=/job:localhost/replica:0/task:0)


In [11]:
train_dataset

<PrefetchDataset element_spec=(TensorSpec(shape=<unknown>, dtype=tf.float32, name=None), (TensorSpec(shape=<unknown>, dtype=tf.float32, name=None), TensorSpec(shape=<unknown>, dtype=tf.float32, name=None)))>

In [12]:
embed_dim = example.shape[-1]
output_dim = embed_dim
encode_dim = 180

In [16]:
num_ae_encoder_layers = 1
num_ae_decoder_layers = 3
num_lstm_decoder_layers = 3

In [17]:
ff_dim = 100  # Hidden layer size in feed forward network inside transformer
dropout_rate = 0.0

inputs = layers.Input(shape=(input_window,embed_dim))
ae_encoding_layers = []
ae_encoding_layers.append(layers.LSTM(encode_dim,activation='elu'))
    
# decoder for reconstruction
ae_decoding_layers = []
ae_decoding_layers.append(layers.RepeatVector(input_window))
ae_decoding_layers.append(layers.LSTM(100,activation='elu', return_sequences=True))
ae_decoding_layers.append(layers.TimeDistributed(layers.Dense(embed_dim)))

# decoder for prediction    
lstm_decoding_layers = []
lstm_decoding_layers.append(layers.RepeatVector(output_window))
lstm_decoding_layers.append(layers.LSTM(100,activation='elu', return_sequences=True))
lstm_decoding_layers.append(layers.TimeDistributed(layers.Dense(embed_dim)))


# Encode from physical space
print('Input shape:',inputs.get_shape().as_list())

x = inputs
for i in range(num_ae_encoder_layers):
    x = ae_encoding_layers[i](x)
encoded = x

print('AE Encoded shape:',encoded.get_shape().as_list())

preds = encoded
for i in range(num_lstm_decoder_layers):
    preds = lstm_decoding_layers[i](preds)

print('LSTM Decoded shape:',preds.get_shape().as_list())
    
recon = encoded
for i in range(num_ae_decoder_layers):
    recon = ae_decoding_layers[i](recon)
    
print('AE Output shape:',recon.get_shape().as_list())

model = tf.keras.Model(inputs=inputs, outputs=[preds,recon])

Input shape: [None, 14, 38979]
AE Encoded shape: [None, 180]
LSTM Decoded shape: [None, 7, 38979]
AE Output shape: [None, 14, 38979]


In [18]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=5, min_lr=0.0001)
early_stop = EarlyStopping(monitor='val_loss',patience=20)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),loss='mean_squared_error',loss_weights=[1,1])

In [19]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 14, 38979)]  0           []                               
                                                                                                  
 lstm_6 (LSTM)                  (None, 180)          28195200    ['input_3[0][0]']                
                                                                                                  
 repeat_vector_5 (RepeatVector)  (None, 7, 180)      0           ['lstm_6[0][0]']                 
                                                                                                  
 repeat_vector_4 (RepeatVector)  (None, 14, 180)     0           ['lstm_6[0][0]']                 
                                                                                              

In [ ]:
history = model.fit(train_dataset,epochs=250,callbacks=[reduce_lr,early_stop])

Epoch 1/250


2022-10-15 16:49:05.613407: W tensorflow/core/common_runtime/forward_type_inference.cc:231] Type inference failed. This indicates an invalid graph that escaped type checking. Error message: INVALID_ARGUMENT: expected compatible input types, but input 1:
type_id: TFT_OPTIONAL
args {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_TENSOR
    args {
      type_id: TFT_BOOL
    }
  }
}
 is neither a subtype nor a supertype of the combined inputs preceding it:
type_id: TFT_OPTIONAL
args {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_TENSOR
    args {
      type_id: TFT_LEGACY_VARIANT
    }
  }
}

	while inferring type of node 'mean_squared_error_1/cond/output/_511'


25/25 [==============================] - 46s 2s/step - loss: 1.0427 - time_distributed_4_loss: 0.3325 - time_distributed_3_loss: 0.7102 - lr: 1.0000e-04
Epoch 2/250
22/25 [=========================>....] - ETA: 3s - loss: 0.3921 - time_distributed_4_loss: 0.1833 - time_distributed_3_loss: 0.2088